In [57]:
import pandas as pd
import requests
import json
!pip install urllib3
import urllib.request
import urllib.parse
import urllib.error

You should consider upgrading via the 'c:\users\moriy\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


In [71]:
url_data = 'C:\\Users\\moriy\\OneDrive - Ariel University\\sofware\\Python_mining_and_analysis\\dests_matala3.txt'
# טעינת היעדים מהקובץ
with open(url_data, 'r') as file:
    destinations = [line.strip() for line in file if line.strip()]
destinations

['Istanbul', 'Amsterdam', 'Valletta', 'Basel', 'Doha']

##### הנחתי שפה זה בסדר להשתמש בלולאה, מאחר ואין כמעט דאטה בקובץ, זו 'רק' רשימת ערים.

In [79]:
# יצירת הדאטה פריים כפי שביקשו
df = pd.DataFrame(columns=['Target', 'Distance_km', 'Duration', 'Longitude', 'Latitude'])

address = 'Tel Aviv'
api_key = 'XXX'

# פונקציה לשליפת מידע על היעד משירות ה-Geocode
def get_geocode_data(address):
    url = 'https://maps.googleapis.com/maps/api/geocode/json?address=%s&key=%s' % (address, api_key)
    params = {
        'address': address,
        'key': api_key
    }
    try:
        response = requests.get(url, params = params)
        if response.status_code == 200:
            data = response.json()
            location = data['results'][0]['geometry']['location']
            return location['lat'], location['lng']
        else:
            print('Geocode request failed:', response.status_code)
            return None, None
    except requests.exceptions.RequestException as e:
        print('An error occurred:', e, '1')
        return None, None
    
# פונקציה לשליפת מידע על המרחק והזמן בין שני יעדים משירות ה-Distance Matrix
def get_distance_data(origin, destination):
    url = 'https://maps.googleapis.com/maps/api/distancematrix/json?address=%s&key=%s' % (address, api_key)
    params = {
        'origins': origin,
        'destinations': destination,
        'key': api_key  # Replace with your actual API key
    }
    try:
        response = requests.get(url, params = params)
        if response.status_code == 200:
            data = response.json()
            distance = data['rows'][0]['elements'][0]['distance']['text']
            duration = data['rows'][0]['elements'][0]['duration']['text']
            return distance, duration
        else:
            print('Distance Matrix request failed:', response.status_code)
            return None, None
    except requests.exceptions.RequestException as e:
        print('An error occurred:', e, '2')
        return None, None

# נעבור על רשימת היעדים
for dest in destinations:
    try:
        # שליפת היעד
        latitude, longitude = get_geocode_data(dest)
        # בדיקה האם נתוני ה-Geocode זמינים
        if latitude is None or longitude is None:
            print('Geocode data not available for', dest)
            continue
        # שליפת מידע על המרחק ועל הזמן בין תל אביב לבין היעד
        distance, duration = get_distance_data('Tel Aviv', dest)
        # המרת הזמן לפורמט שעות ודקות בלבד
        hours = 0
        minutes = 0
        duration_parts = duration.split(' ')
        if duration_parts[1] == 'days' or duration_parts[1] == 'day':
            hours = int(duration_parts[0]) * 24 + int(duration_parts[2])
            minutes = 0
        else:
            hours = int(duration_parts[0])
            minutes = int(duration_parts[2])
        duration_formatted = f'{hours} hours {minutes} minutes'
        new_data = pd.DataFrame({
            'Target': [dest],
            'Distance_km': [distance],
            'Duration': [duration_formatted],
            'Longitude': [longitude],
            'Latitude': [latitude]
        })
        df = pd.concat([df, new_data], ignore_index=True)
    except Exception as e:
        print('An error occurred:', e, '3')

In [80]:
df

,Target,Distance_km,Duration,Longitude,Latitude
0,Istanbul,"1,815 km",21 hours 3 minutes,28.978359,41.008238
1,Amsterdam,"4,533 km",48 hours 0 minutes,4.904139,52.367573
2,Valletta,"3,793 km",51 hours 0 minutes,14.514100,35.899237
3,Basel,"4,118 km",44 hours 0 minutes,7.588576,47.559599
4,Doha,"2,164 km",22 hours 39 minutes,51.531040,25.285447


In [81]:
# המרת העמודה "Distance_km" למספר
df['Distance_km'] = df['Distance_km'].str.extract('(\d+\.?\d*)').astype(float)
# 3 הערים הרחוקות ביותר מתל אביב
furthest_cities = df.nlargest(3, 'Distance_km')
furthest_cities

,Target,Distance_km,Duration,Longitude,Latitude
1,Amsterdam,4.0,48 hours 0 minutes,4.904139,52.367573
3,Basel,4.0,44 hours 0 minutes,7.588576,47.559599
2,Valletta,3.0,51 hours 0 minutes,14.514100,35.899237
